In [1]:
using Pkg
Pkg.activate("./")
include("./src3/include.jl")

  Activating project at `~/code/Surreal.jl`


In [152]:
"""
0: not defined / empty set
positive: index in the surreal cache
negative: index in the expressions cache
"""
const Index = Int64

struct SurrealExpression
    # TODO
    expr::Any
end

struct SurrealContext{T}
    "list of surreals in use with their information. numbers are never deleted to make sure they remain compatible"
    cache::Vector{T} 
    
    "list of surreal expressions used by numbers"
    expressions::Vector{SurrealExpression}
    
    "was copied from this one. Since the cache cannot be deleted, we can use numbers from the parent without the costly process of importing"
    parent::Union{Nothing,SurrealContext{T}}
    
    "find Surreal by anchestor"
    byAnchestor::Dict{Tuple{Index, Index}, Index}
    
    "find Surreal by addends"
    byAddend::Dict{Tuple{Index, Index}, Index}
        
    function SurrealContext{T}() where {T}
        new{T}([], [], nothing, Dict(), Dict())
    end
end
    

struct SurrealStatic
    # index of left set or surreal
    left::Index
    
    # index of right set or surreal
    right::Index
    
    #canonical::Index
    #birthday::Index
        
    # optional information, such as "isFinite"
    flags1::UInt64
    flags2::UInt64

    function SurrealStatic(left::Index, right::Index)
        new(left,right,0,0)
    end
end

const Ctx = SurrealContext{SurrealStatic}

# TODO: cache it
"might implicitly modify ctx"
zero(ctx::Ctx) = Surreal!(ctx, nothing, nothing)
    
# TODO: cache it
"might implicitly modify ctx"
one(ctx::Ctx) = Surreal!(ctx, zero(ctx), nothing)

# TODO: cache it
"might implicitly modify ctx"
minusone(ctx::Ctx) = Surreal!(ctx, nothing, zero(ctx))
    
@assert isbits(SurrealStatic(0,0))
    
abstract type Surreal end
struct SurrealCached <: Surreal
    ctx::Ctx
    index::Index
    function SurrealCached(ctx::Ctx, index::Index)
        @assert index > 0 "must be a number index and not a set index"
        @assert index <= length(ctx.cache) "number is not found in this cache"
        new(ctx, index)
    end
end
Surreal(ctx::Ctx, index::Index) = SurrealCached(ctx, index)

function findOrCreateSurreal!(ctx::Ctx, left::Index, right::Index)
    if haskey(ctx.byAnchestor, (left, right))
        return ctx.byAnchestor[(left, right)]  
    end
        
    # TODO: assert left < right
        
    push!(ctx.cache, SurrealStatic(left, right))
    local l = length(ctx.cache)
    ctx.byAnchestor[(left, right)] = l
    return l 
end

Surreal!(ctx::Ctx, ::Nothing, ::Nothing) = SurrealCached(ctx, findOrCreateSurreal!(ctx, 0, 0))

"mighty implicitly modify the ctx"
Surreal(left::SurrealCached, ::Nothing) = SurrealCached(left.ctx, findOrCreateSurreal!(left.ctx, left.index, 0))

"mighty implicitly modify the ctx"
Surreal(::Nothing, right::SurrealCached) = SurrealCached(right.ctx, findOrCreateSurreal!(right.ctx, 0, right.index))

"mighty implicitly modify the ctx"
function Surreal(left::SurrealCached, right::SurrealCached)
    # TODO: 
    # if x and y have different tables:
    # -> if one is parent: use the child one for the result
    # -> if they are incompatible: copy everything to the left one
    @assert left.ctx === right.ctx
        
    local ctx = left.ctx
    SurrealCached(ctx, findOrCreateSurreal!(ctx, left.index, right.index))
end

function Surreal!(ctx::Ctx, n::Int64)
    if n == 0
        return zero(ctx)
    elseif n > 0
        return Surreal(Surreal!(ctx, n - 1), nothing)
    end
    @assert n < 0
    return Surreal(nothing, Surreal!(ctx, n + 1))
end
        
struct SurrealDyadic{T} <: Surreal
    val::T
end

Surreal(x::Rational) = SurrealDyadic{Rational{BigInt}}(x)
Surreal(x::BigInt) = SurrealDyadic{BigInt}(x)
Surreal(x::Int64) = SurrealDyadic{Int64}(x)

Surreal

In [153]:
ctx = SurrealContext{SurrealStatic}()
Surreal!(ctx, -4)
ctx.cache

5-element Vector{SurrealStatic}:
 SurrealStatic(0, 0, 0x0000000000000000, 0x0000000000000000)
 SurrealStatic(0, 1, 0x0000000000000000, 0x0000000000000000)
 SurrealStatic(0, 2, 0x0000000000000000, 0x0000000000000000)
 SurrealStatic(0, 3, 0x0000000000000000, 0x0000000000000000)
 SurrealStatic(0, 4, 0x0000000000000000, 0x0000000000000000)

In [154]:
f(ctx::Ctx) = begin
    @inbounds ctx.cache[2].left
end

f(ctx)
@code_llvm  f(ctx)

;  @ In[154]:1 within `f`
define i64 @julia_f_3052({}* noundef nonnull readonly align 8 dereferenceable(40) %0) #0 {
top:
;  @ In[154]:2 within `f`
; ┌ @ Base.jl:37 within `getproperty`
   %1 = bitcast {}* %0 to [4 x i64]***
   %2 = load atomic [4 x i64]**, [4 x i64]*** %1 unordered, align 8
; └
; ┌ @ essentials.jl:13 within `getindex`
   %3 = load [4 x i64]*, [4 x i64]** %2, align 8
   %.sroa.0.0..sroa_idx = getelementptr inbounds [4 x i64], [4 x i64]* %3, i64 1, i64 0
   %.sroa.0.0.copyload = load i64, i64* %.sroa.0.0..sroa_idx, align 1
; └
  ret i64 %.sroa.0.0.copyload
}


In [155]:
@inline left(ctx::Ctx, index::Index)::Index = ctx.cache[index].left
@inline right(ctx::Ctx, index::Index)::Index = ctx.cache[index].right
@inline left(x::SurrealCached)::Index = left(x.ctx,x.index)
@inline right(x::SurrealCached)::Index = right(x.ctx,x.index)

function Base.max(ctx::Ctx, x::Index, y::Index)
    x == y && return x
    TODO
end

function Base.min(ctx::Ctx, x::Index, y::Index)
    x == y && return x
    TODO
end

"modifies ctx"
function add(ctx::Ctx, x::Index, y::Index)
    
    @assert x > 0 "must be a number"
    @assert y > 0 "must be a number"
    
    zero(ctx).index == x && return y
    zero(ctx).index == y && return x
    
    local k = (min(x,y), max(x,y))
    if haskey(ctx.byAddend, k)
        return ctx.byAddend[k]
    end
    
    #=
    TODO: Canonicaclize early, i.e.,
    for i in -1000:1000
        local res = Surreal!(ctx, i) + (1 + Surreal(10)) + Surreal!(ctx, -i)
        res.index < 0 && return false          
    end
    generates over 1 million temporary non-canonical intermediate values
    =#
    
    local xl = left(ctx, x)
    local yl = left(ctx, y)
    local xr = right(ctx, x)
    local yr = right(ctx, y)
    
    @assert xl >= 0 "TODO: expressions not supported"
    @assert yl >= 0 "TODO: expressions not supported"
    @assert xr >= 0 "TODO: expressions not supported"
    @assert yr >= 0 "TODO: expressions not supported"

    local l = xl == 0 ? 0 : add(ctx, xl, y)
    if yl > 0
        local tmp = add(ctx, yl, x)
        l = l == 0 ? tmp : max(ctx, l, tmp)
    end
    
    local r = xr == 0 ? 0 : add(ctx, xr, y)
    if yr > 0
        local tmp = add(ctx, yr, x)
        r = r == 0 ? tmp : min(ctx, r, tmp)
    end
        
    local res = findOrCreateSurreal!(ctx, l, r)    
    ctx.byAddend[k] = res
    #@show x,y, l,r, res
    return res    
end

function Base.:(+)(x::SurrealCached, y::SurrealCached)
    # TODO: also handle when the contexts are different
    @assert x.ctx === y.ctx
    local ctx = x.ctx
    SurrealCached(ctx, add(ctx, x.index, y.index))
end

"add them naively, without using any surreal context"
function Base.:(+)(x::SurrealDyadic{T}, y::SurrealDyadic{S}) where {T, S}
    TODO
end

can_safe_add(n1::T, n2::T) where {T<:Integer} = ((n2 > 0) ? (n1 > (typemax(T) - n2)) : (n1 < (typemin(T) - n2))) ? false : true
can_safe_mul(n1::T, n2::T) where {T<:Integer} = ((n2 > 0) ? ((n1 > div(typemax(T),n2)) || (n1 < div(typemin(T),n2))) :
                                      (n2 <  -1) ? ((n1 > div(typemin(T),n2)) || (n1 < div(typemax(T),n2))) :
                                      ((n2 == -1) && n1 == typemin(T))) ? false : true

function Base.:(+)(x::SurrealDyadic{Int64}, y::SurrealDyadic{Int64})
    can_safe_add(x.val, y.val) ? Surreal(x.val+y.val) : Surreal(big(x.val)+big(y.val))
end

function Base.:(+)(x::SurrealCached, y::SurrealDyadic{T}) where {T} 
   x + Surreal!(x.ctx, y.val)
end

Base.:(+)(x::Surreal, y::Int64) = x + Surreal(y)
Base.:(+)(x::Int64, y::Surreal) = Surreal(x) + y

ctx = SurrealContext{SurrealStatic}()
f(ctx::Ctx) = for i in -100:100
    local res = Surreal!(ctx, i) + (1 + Surreal(10)) + Surreal!(ctx, -i)
    res.index < 0 && return false          
end
@time f(ctx)
@show length(ctx.cache)

  0.018356 seconds (239.50 k allocations: 15.346 MiB)
length(ctx.cache) = 11312


11312